# Data Loader- Dataset Types 

讀取檔案的function常稱呼為Data Loader。Tensorflow 2.0本身帶有一些程式，可以客製化的讀取檔案，在做Deep Learning時將資料從CPU或硬碟讀取到GPU，使資料能夠與Model參數在GPU上運算。
TF2.0中 tf.data這個庫中的各種class和function就是來幫助各位客製化data loader。

**Data Loader要件:**
1. Dataset物件- 抽取data的來源，可以用各種方式形成，

   可以被iterate(e.g. for x in Dataset)，但不可以被index (e.g. Dataset[3])
2. Dataset操作- 對Dataset以特定方式抽取、分割、合併、加速、映射等等動作，來行程最後訓練、測試用的Data Loader

這邊我們先從各種Dataset type開始介紹起

## 內容:
1. From Tensors
2. From Tensor Slices
3. List Files
4. From Generator

In [8]:
import tensorflow as tf
from tensorflow import data
import numpy as np

## From Tensors

從tensor建立只有一單位資料的Dataset: .from_tensors

In [43]:
dataset=data.Dataset.from_tensors([0,1,2])
for i in dataset:
    print(i)

tf.Tensor([0 1 2], shape=(3,), dtype=int32)


該Dataset只有一個項目，要透過後續操作把項目依照指定方式分開才能後續使用，實際上有點冗贅

使用起來跟tf.constant沒什麼差別

# From Tensor Slices

從tensor建立有數個單位資料的Dataset: .from_tensor_slices

In [44]:
dataset=data.Dataset.from_tensor_slices([0,1,2])
for i in dataset:
    print(i)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)


會依照輸入項目轉成Tensor，再由第0 rank分裂成數個單位，在iterate時會一個個輸出

## List Files

用一列檔案名稱建立Dataset: .list_files

In [45]:
dataset=data.Dataset.list_files("example_data/*.txt")
for i in dataset:
    print(i)

tf.Tensor(b'example_data/0.txt', shape=(), dtype=string)
tf.Tensor(b'example_data/1.txt', shape=(), dtype=string)
tf.Tensor(b'example_data/2.txt', shape=(), dtype=string)
tf.Tensor(b'example_data/3.txt', shape=(), dtype=string)


這種Dataset要求輸入的是file_pattern，可以包含萬用符。
若將檔案化為一個List來做.list_files會變慢

要注意的是，若列舉檔案時要保持檔案順訊，記得要加shuffle=False

In [54]:
dataset=data.Dataset.list_files("example_data/*.txt",shuffle=False)
for i in dataset:
    print(i)

tf.Tensor(b'example_data/0.txt', shape=(), dtype=string)
tf.Tensor(b'example_data/1.txt', shape=(), dtype=string)
tf.Tensor(b'example_data/2.txt', shape=(), dtype=string)
tf.Tensor(b'example_data/3.txt', shape=(), dtype=string)


In [56]:
i.dtype

tf.string

## From Generator

從function建立Dataset: .from_generator

要先建立一個function，且必須要是能iterate的funciton

In [60]:
def f(rng):
    for i in range(rng):
        yield i

In [48]:
dataset=data.Dataset.from_generator(f,output_types=tf.float32,args=[3])
for i in dataset:
    print(i)

tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)


非常活的一種Dataset，可以指定:
1. generator: 使用的funciton，input是什麼output是什麼都可以自己訂，只要是一個iterable function
2. output_types: 輸出的資料型別，會是tensor
3. (optional)args: 代入的arguement
4. (optional)output_shape: output應該什麼形狀，可供檢查用

就算輸出是string也可以，只要output_type有對上就好

In [68]:
def f(rng):
    for i in range(rng):
        yield str(i)

In [69]:
dataset=data.Dataset.from_generator(f,output_types=tf.string,args=[3])
for i in dataset:
    print(i)

tf.Tensor(b'0', shape=(), dtype=string)
tf.Tensor(b'1', shape=(), dtype=string)
tf.Tensor(b'2', shape=(), dtype=string)


如果function是無窮迴圈也可以，但要在後續操作設立停止條件

In [77]:
def f():
    i=0
    while 1:
        i+=1
        yield i

In [80]:
dataset=data.Dataset.from_generator(f,output_types=tf.float32)
for e,i in enumerate(dataset):
    print(e,i)
    if e>=10:
        break

0 tf.Tensor(1.0, shape=(), dtype=float32)
1 tf.Tensor(2.0, shape=(), dtype=float32)
2 tf.Tensor(3.0, shape=(), dtype=float32)
3 tf.Tensor(4.0, shape=(), dtype=float32)
4 tf.Tensor(5.0, shape=(), dtype=float32)
5 tf.Tensor(6.0, shape=(), dtype=float32)
6 tf.Tensor(7.0, shape=(), dtype=float32)
7 tf.Tensor(8.0, shape=(), dtype=float32)
8 tf.Tensor(9.0, shape=(), dtype=float32)
9 tf.Tensor(10.0, shape=(), dtype=float32)
10 tf.Tensor(11.0, shape=(), dtype=float32)
